In [1]:
import pandas as pd
import json

In [2]:
data = []
with open('./tmdbKr.json','r',encoding='utf8') as file :
    data= json.load(file)

In [3]:
# 빈 데이터프레임
df = pd.DataFrame(columns=['title','genres','id','overview','popularity','original_title','vote_average','vote_count'])

In [4]:
df

,title,genres,id,overview,popularity,original_title,vote_average,vote_count


In [5]:
# 영화데이터

# 모든데이터 for문 돌면서
# model == movies.movie 일때
# 각 값들을 뽑아서 새로운 데이터프레임으로 만들고 
# 기존 데이터프레임에 합쳐주는 과정
genre_list = list()
for i in range(len(data)) :
    if data[i]['model'] == 'movies.genre':
        genre_list.append(data[i])
    elif data[i]['model']=='movies.movie':
        title =  data[i]['fields']['title']
        gen = data[i]['fields']['genre_ids']
        #print(gen)
        #break
        gen_id2str = list()
        for gen_ in gen:
            for _a in genre_list:
                if _a['pk'] == gen_:
                    gen_id2str.append(_a['fields']['name'])
                    break    
        
        #print(gen_id2str)
        #break
            
        
        #gen_str = ','.join(str(e) for e in gen) # 여러값이 있어서 해야하는데 이해안감
        id = data[i]['pk']
        overview = data[i]['fields']['overview']
        popularity = data[i]['fields']['popularity']
        original_title = data[i]['fields']['original_title']
        vote_average = data[i]['fields']['vote_average']
        vote_count = data[i]['fields']['vote_count']

        data_json = {'title':[title],'genres':[gen_id2str],'id':[id],'overview':[overview],'popularity':[popularity],'original_title':[original_title],'vote_average':[vote_average],'vote_count':[vote_count]}
        df_tmp= pd.DataFrame(data_json)
        df =  pd.concat([df, df_tmp], ignore_index = True)

In [7]:
df.head(15)

,title,genres,id,overview,popularity,original_title,vote_average,vote_count
0,앤트맨과 와스프: 퀀텀매니아,"[액션, 모험, SF]",640146,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",10773.574,Ant-Man and the Wasp: Quantumania,6.5,1999
1,슈퍼 마리오 브라더스,"[애니메이션, 모험, 가족, 판타지, 코미디]",502356,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,8786.695,The Super Mario Bros. Movie,7.5,1580
2,샤잠! 신들의 분노,"[액션, 코미디, 판타지]",594767,신들의 힘을 갖게 된 빌리(애셔 앤젤)와 친구들은 각자의 방법으로 슈퍼히어로의 삶을...,5759.074,Shazam! Fury of the Gods,6.9,1336
3,아바타: 물의 길,"[SF, 모험, 액션]",76600,판도라 행성에서 제이크 설리와 네이티리가 이룬 가족이 겪게 되는 무자비한 위협과 살...,4557.845,Avatar: The Way of Water,7.7,7644
4,세븐 킹스 머스트 다이,"[액션, 모험, 역사, 드라마, 전쟁]",948713,에드워드 왕이 서거하자 뜻을 모으는 베반버그의 우트레드와 그의 동지들. 마침내 잉글...,3903.799,The Last Kingdom: Seven Kings Must Die,7.3,263
5,크리드 3,"[드라마, 액션]",677179,록키의 후계자인 크리드는 월드 챔피언으로 권투계를 장악한 후 완벽한 삶을 살던 중 ...,3546.342,Creed III,7.3,1228
6,머더 미스터리 2,"[코미디, 미스터리, 액션]",638974,결혼식에서 억만장자 신랑이 납치당한 사건. 탐정 사무소를 차린 닉과 오드리 스피츠에...,2464.707,Murder Mystery 2,6.6,905
7,고스팅,"[로맨스, 액션, 코미디]",868759,선량한 남자 콜은 수수께끼 같은 세이디에게 푹 빠지지만 곧 그녀가 비밀 요원이라는 ...,2381.140,Ghosted,7.3,461
8,스크림 6,"[공포, 미스터리, 스릴러]",934433,,2343.149,Scream VI,7.4,841
9,이블 데드 라이즈,"[공포, 스릴러]",713704,길바닥에서 지친 베스가 비좁은 아파트에서 세 아이를 키우는 엘리를 뒤늦게 찾아간다....,2192.777,Evil Dead Rise,7.0,270


In [17]:
df.to_csv('./tmdbData.csv',encoding='utf8',index=False)

## 영화 검색

In [ ]:
from tmdbv3api import TMDb, Movie

tmdb = TMDb()
tmdb.api_key = "234c446afa7c9156e6c9a101b724f5ae"

movie = Movie()

m = movie.details(111)

print(m.title)
print(m.overview)
print(m.popularity)


Scarface
After getting a green card in exchange for assassinating a Cuban government official, Tony Montana stakes a claim on the drug trade in Miami. Viciously murdering anyone who stands in his way, Tony eventually becomes the biggest drug lord in the state, controlling nearly all the cocaine that comes through Miami. But increased pressure from the police, wars with Colombian drug cartels and his own drug-fueled paranoia serve to fuel the flames of his eventual downfall.
83.822


In [ ]:
# 영화 검색
import requests


def SearchMovies(query, year):  # 영화 제목과 년도 정보를 받음

    params = {"api_key": "234c446afa7c9156e6c9a101b724f5ae",
          "language": "ko",
          "page": "1",
          "query": query,
          "include_adult": "true",
          "year": year}


    url = "https://api.themoviedb.org/3/search/movie" # 정보를 요청할 주소
    resp = requests.get(url, params=params)
    data = resp.json()['results']


    if(len(data) > 0): # 검색해서 나온 데이터가 있을 경우에만

       for item in data : # 반복문 돌면서 영화 이름과 제작년도 출력
           print(item['original_title'], item['release_date']) 

if __name__ == '__main__':
    SearchMovies('존 윅',"2023")

John Wick: Chapter 4 2023-03-22
